# Importaciones
Primero tenemos que importar las librerias a usar.
- Numpy : libreria numerica de matrices.
- matplotlib: libreria para graficaion
- Pandas: libreria para manejar tablas
- Tabulate: extension de tablas

In [7]:
import pandas as pn
import numpy as np
import matplotlib as plt
import tabulate as tabl
from typing import Any

# Parte 1.
Para implementar el método simplex tenemos que introducir los "elementos principales":

<figure>
    <br>
    <center> <img src="./images/elementos.png"   style="width:500px;height:280px;" ></center>
</figure>

En este codigo dividiermos todo en 3 elementos:
1. $\LARGE a\_i$: Va a ser la matriz que contiene todos los coeficientes de las restricciones, de la forma $\large [A,I]$
2. $\LARGE b$ es el vector con los valores de las soluciones de las restricciones.
3. $\LARGE c$ es el vector con los coeficientes de las variables en la F.O.


In [8]:
a_i : np.ndarray[np.int_] = np.array([[-2,1,1,0,0],
                                      [-1,2,0,1,0],
                                      [1,0,0,0,1]])
b  : np.ndarray[np.int_] = np.array([2,7,3])
c  : np.ndarray[np.int_] = np.array([-1,-2,0,0,0])

# 2. Ahora debemos de implementar simplex.

In [9]:
# TODO: Actualmente el código lo mantengo con type: np.int_ para reducir errores. Una vez listo hay que cambiar el tipo para permitir float
def Simplex(
    A: np.ndarray[np.int_], b: np.ndarray[np.int_], c: np.ndarray[np.int_]
) -> Any:
    """
    Le pasamos los vectores con los datos del ejercicio. Utiliza un while loop para iterar hasta que se encuentre la solución óptima
    :param A: Matriz con coeficientes de las restricciones :type A: np.ndarrya[np.int_]
    :param b: Matriz con los valores de las soluciones de las restriciones :type  b: np.ndarrya[np.int_]:
    :param c: Matriz con los coeficientes de la F.O. :type c: np.ndarrya[np.int_]
    :return:
    """

    tamano_basico = A.shape[0]  # number of constraints, m
    tamano_noBasico = A.shape[1] - tamano_basico  # n-m, number of variables

    # Tenemos un problema ahora, las variables del array C esta escrito de la forma X_0, X_1, X_2, por lo tanto esta linea de código guardará los índices:
    indice_c: list[int] = [i for i in range(0, len(c))]

    # Coeficientes de las variables basicas y no báscias:
    coef_vBasicas: np.ndarray[np.int_] = np.array(c[tamano_noBasico:])
    coef_vNBasicas: np.ndarray[np.int_] = np.array(c[:tamano_noBasico])

    # Corrida en frio de prueba para revisar que el programa plantee bien el problema. Comentar si no es necesario.
    print(f"La cantidad de condiciones son: {tamano_basico}")
    print(f"La cantidad de variables son: {tamano_noBasico}\n")
    print(f"Los coeficientes de las variables basicas son: {coef_vBasicas} \n")
    print(f"Los coeficientes de als variables NO basicas son: {coef_vNBasicas}\n")

    while True:
        # Mantenemos los índices de las variables básicas y no-básicas aquí
        indice_actual_basica = indice_c[tamano_noBasico:]
        indice_actual_noBasica = indice_c[:tamano_noBasico]

        "Matriz Básica:"
        B = A[:, indice_actual_basica]
        B_inversa = np.linalg.inv(B)

        "Matriz de variables no básicas:"
        N = A[:, indice_actual_noBasica]

        "bHat son los valores de las variables basicas, con su resultao b"
        bHat = B_inversa @ b
        yT = coef_vBasicas @ B_inversa

        "La siguiente linea de codigo busca optimizar, determina cual variable va a entrar"
        cnHat = coef_vNBasicas - (yT @ N)

        # Encuentra el índice cn el valor minimo de cnHat, esta es la variable que va a entrar
        cn_indice_minimo = np.argmin(cnHat)

        """La siguiente linea revisa si se encontró el mino, si es asi. Rompe el ciclo While True:
        
        Utiliza indice_actual_basica para obtener los indices de las variables+indices en bHat. 
        Los valores en bHat son la solución final para cada una de las variables correspondientes.
        """
        if all(i >= 0 for i in cnHat):
            return (
                coef_vBasicas,
                indice_actual_basica,
                coef_vNBasicas,
                indice_actual_noBasica,
                bHat,
                cnHat,
            )

        # Este es el índice para la columna de coeficientes dado su respectiva variable
        indice = indice_c[cn_indice_minimo]
        Ahat = B_inversa @ A[:, indice]

        # El siguiente código itera a travez de Ahat y bHat, y va a agarrar los valores mínimos siempre y cuando sean mayores a 0.
        # El valor más pequeño mayor a cero, será la variable que se transformara en no-básica
        ratios = []
        for _ in range(0, len(bHat)):
            Aval = Ahat[_]
            Bval = bHat[_]

            # Este filtro es para no ver a los "ratio" con valor menor o igual a cero, se anexará para mantener el índice.
            if Aval <= 0:
                ratios.append(10000000)
                continue
            ratios.append(Bval / Aval)
        # Se obtiene el valor minio:
        ratio_menor_indice = np.argmin(ratios)

        # Intercambiamos las variables básicas y no básicas usando los índices
        coef_vNBasicas[cn_indice_minimo], coef_vBasicas[ratio_menor_indice] = (
            coef_vBasicas[ratio_menor_indice],
            coef_vNBasicas[cn_indice_minimo],
        )
        # Intercambiamos el indice global, y el trackeador de índice:
        indice_c[cn_indice_minimo], indice_c[ratio_menor_indice + tamano_noBasico] = (
            indice_c[ratio_menor_indice + tamano_noBasico],
            indice_c[cn_indice_minimo],
        )
        # Se repite el loop.


In [10]:
Simplex(a_i,b,c)

La cantidad de condiciones son: 3
La cantidad de variables son: 2

Los coeficientes de las variables basicas son: [0 0 0] 

Los coeficientes de als variables NO basicas son: [-1 -2]


(array([-2, -1,  0]),
 [1, 0, 2],
 array([0, 0]),
 [3, 4],
 array([5., 3., 3.]),
 array([1., 2.]))